In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import statsmodels.api as sm



/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# OAI dataset
df = pd.read_csv("preprocessed_OAI.csv")
# GLAD dataset
dfg = pd.read_csv("preprocessed_Glad.csv")
df.shape
print("OAI shape:", df.shape, "GLAD shape:", dfg.shape)

OAI shape: (4131, 11) GLAD shape: (7603, 11)


In [3]:
df

,V00AGE,P02SEX,P01BMI,V00CESD6,V00CEMPLOY_employment,PA_baseline_category,PA_followup_category,V00KOOSQOL,V01KOOSQOL,knee_pain_baseline,knee_pain_follow
0,78.0,1.0,27.6,1.0,3.0,1,1,50.00,68.75,8.0,5.0
1,76.0,2.0,27.4,1.0,3.0,1,3,100.00,62.50,0.0,4.0
2,69.0,1.0,29.8,1.0,3.0,1,4,81.25,100.00,1.0,0.0
3,68.0,2.0,30.1,2.0,3.0,1,1,75.00,81.25,3.0,3.0
4,76.0,1.0,32.4,1.0,2.0,1,1,75.00,68.75,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,10,9,100.00,100.00,0.0,0.0
4127,59.0,2.0,30.5,1.0,1.0,10,6,43.75,68.75,7.0,7.0
4128,70.0,1.0,33.9,1.0,3.0,10,5,100.00,100.00,0.0,0.0
4129,52.0,1.0,30.7,1.0,1.0,10,8,31.25,37.50,5.0,7.0


In [4]:
dfg

,age,gender,fysb_BMI,ptb_14618_depression,ptb_3777_employment,ptb_4145,pt12_4145,ptb_koos_qol_score,pt12_koos_qol_score,ptb_3764,pt12_3764
0,50,2,22.68,1.0,1.0,10.0,10.0,31.25,56.25,65.0,17.0
1,45,1,29.07,1.0,1.0,7.0,10.0,25.00,43.75,96.0,52.0
2,46,2,26.10,1.0,1.0,3.0,9.0,68.75,62.50,39.0,25.0
3,53,2,28.03,1.0,1.0,4.0,4.0,62.50,87.50,15.0,2.0
4,68,2,27.82,1.0,1.0,8.0,10.0,87.50,93.75,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
7598,55,2,35.63,1.0,1.0,8.0,10.0,50.00,56.25,57.0,9.0
7599,61,2,29.34,1.0,1.0,3.0,6.0,25.00,68.75,55.0,8.0
7600,62,1,25.88,1.0,1.0,3.0,3.0,50.00,25.00,26.0,77.0
7601,47,2,26.51,1.0,1.0,4.0,2.0,43.75,18.75,47.0,92.0


In [5]:
# Renaming columns in df for consistency
df.rename(columns={
    'V00AGE': 'Age',
    'P02SEX': 'Gender',
    'P01BMI': 'BMI',
    'V00CESD6': 'Depression',
    'V00CEMPLOY_employment': 'Employment_status',
    'V00KOOSQOL': 'Baseline_Quality_of_life',
    'V01KOOSQOL': 'Followup_Quality_of_life',
    'knee_pain_baseline': 'Baseline_Pain',
    'knee_pain_follow': 'Followup_Pain',
    'PA_baseline_category': 'Baseline_Physical_Activity',
    'PA_followup_category': 'Followup_Physical_Activity'
}, inplace=True)

dfg.rename(columns={
    'age': 'Age',
    'gender': 'Gender',
    'fysb_BMI': 'BMI',
    'ptb_14618_depression': 'Depression',
    'ptb_3777_employment': 'Employment_status',
    'ptb_koos_qol_score': 'Baseline_Quality_of_life',
    'pt12_koos_qol_score': 'Followup_Quality_of_life',
    'ptb_3764': 'Baseline_Pain',
    'pt12_3764': 'Followup_Pain',
    'ptb_4145': 'Baseline_Physical_Activity',
    'pt12_4145': 'Followup_Physical_Activity'
}, inplace=True)


In [6]:
dfg

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Followup_Quality_of_life,Baseline_Pain,Followup_Pain
0,50,2,22.68,1.0,1.0,10.0,10.0,31.25,56.25,65.0,17.0
1,45,1,29.07,1.0,1.0,7.0,10.0,25.00,43.75,96.0,52.0
2,46,2,26.10,1.0,1.0,3.0,9.0,68.75,62.50,39.0,25.0
3,53,2,28.03,1.0,1.0,4.0,4.0,62.50,87.50,15.0,2.0
4,68,2,27.82,1.0,1.0,8.0,10.0,87.50,93.75,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
7598,55,2,35.63,1.0,1.0,8.0,10.0,50.00,56.25,57.0,9.0
7599,61,2,29.34,1.0,1.0,3.0,6.0,25.00,68.75,55.0,8.0
7600,62,1,25.88,1.0,1.0,3.0,3.0,50.00,25.00,26.0,77.0
7601,47,2,26.51,1.0,1.0,4.0,2.0,43.75,18.75,47.0,92.0


In [7]:
df.Depression.unique()

array([1., 2., 3., 4.])

In [8]:
# Define covariates for balancing
covariates = ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 'Baseline_Physical_Activity', 'Baseline_Quality_of_life', 'Baseline_Pain']
X_control = df[covariates].values
X_treatment = dfg[covariates].values

# Calculate the mean values of the covariates in the treatment group
mean_treatment = X_treatment.mean(axis=0)

# Define entropy balancing objective and constraints
def entropy_balancing(weights, X_control, mean_treatment):
    weighted_mean_control = np.average(X_control, axis=0, weights=weights)
    constraint = weighted_mean_control - mean_treatment
    return np.sum(weights * np.log(weights)), constraint

# Initial weights
initial_weights = np.ones(len(X_control)) / len(X_control)

# Constraint function
def constraints(weights):
    return entropy_balancing(weights, X_control, mean_treatment)[1]

# Optimization to find the optimal weights
result = minimize(
    lambda w: entropy_balancing(w, X_control, mean_treatment)[0],
    initial_weights,
    constraints={'type': 'eq', 'fun': constraints},
    bounds=[(0, None)] * len(X_control)
)

optimal_weights = result.x

# Apply the weights to the control group
df['weights'] = optimal_weights

/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_87257/1830198809.py:13: RuntimeWarning: divide by zero encountered in log
  return np.sum(weights * np.log(weights)), constraint
/var/folders/rn/9h9wnd_x13930h39r4t9tt_w0000gr/T/ipykernel_87257/1830198809.py:13: RuntimeWarning: invalid value encountered in multiply
  return np.sum(weights * np.log(weights)), constraint


In [9]:
df

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Followup_Physical_Activity,Baseline_Quality_of_life,Followup_Quality_of_life,Baseline_Pain,Followup_Pain,weights
0,78.0,1.0,27.6,1.0,3.0,1,1,50.00,68.75,8.0,5.0,0.215431
1,76.0,2.0,27.4,1.0,3.0,1,3,100.00,62.50,0.0,4.0,0.029756
2,69.0,1.0,29.8,1.0,3.0,1,4,81.25,100.00,1.0,0.0,0.342891
3,68.0,2.0,30.1,2.0,3.0,1,1,75.00,81.25,3.0,3.0,0.397441
4,76.0,1.0,32.4,1.0,2.0,1,1,75.00,68.75,3.0,4.0,0.089751
...,...,...,...,...,...,...,...,...,...,...,...,...
4126,56.0,1.0,24.1,1.0,1.0,10,9,100.00,100.00,0.0,0.0,0.012374
4127,59.0,2.0,30.5,1.0,1.0,10,6,43.75,68.75,7.0,7.0,0.032593
4128,70.0,1.0,33.9,1.0,3.0,10,5,100.00,100.00,0.0,0.0,0.005297
4129,52.0,1.0,30.7,1.0,1.0,10,8,31.25,37.50,5.0,7.0,0.015927


In [10]:
weighted_mean_physical_activity_control = np.average(df['Physical_Activity'], weights=df['weights'])
print(f"Weighted Mean Physical Activity (Control Group): {weighted_mean_physical_activity_control}")

KeyError: 'Physical_Activity'

In [ ]:
mean_physical_activity_treatment = dfg['Physical_Activity'].mean()
print(f"Mean Physical Activity (Treatment Group): {mean_physical_activity_treatment}")

Mean Physical Activity (Treatment Group): 5.7978429567276075


##### Using Statsmodels for Weighted Linear Regression

In [ ]:

# Define the independent variables (including a constant for the intercept)
X = df[['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 'Quality_of_life', 'Pain']]
X = sm.add_constant(X)

# Define the dependent variable
y = df['Physical_Activity']

# Define the weights
weights = df['weights']

# Fit the weighted least squares regression model
wls_model = sm.WLS(y, X, weights=weights)
results = wls_model.fit()

# Print the summary of the regression results
print(results.summary())


                            WLS Regression Results                            
Dep. Variable:      Physical_Activity   R-squared:                       0.190
Model:                            WLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     139.0
Date:                Thu, 06 Jun 2024   Prob (F-statistic):          1.56e-184
Time:                        16:44:10   Log-Likelihood:                -12449.
No. Observations:                4155   AIC:                         2.491e+04
Df Residuals:                    4147   BIC:                         2.497e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                11.8044      0.37

In [ ]:
weighted_df = df.copy()

# Apply weights to each variable
for column in ['Age', 'Gender', 'BMI', 'Depression', 'Employment_status', 'Quality_of_life', 'Pain', 'Physical_Activity']:
    weighted_df[column] = weighted_df[column] * weighted_df['weights']

# Display the new DataFrame with weighted values


In [ ]:
weighted_df

,Age,Gender,BMI,Depression,Employment_status,Physical_Activity,Quality_of_life,Pain,weights
0,0.031023,0.000398,0.010977,0.000398,0.001193,0.000398,0.019887,0.001641,0.000398
1,0.092652,0.002438,0.033403,0.001219,0.003657,0.001219,0.121911,0.000000,0.001219
2,0.040358,0.000585,0.017430,0.000585,0.001755,0.000585,0.047523,0.001170,0.000585
3,0.041512,0.001221,0.018375,0.001221,0.001831,0.000610,0.045785,0.012095,0.000610
4,0.353639,0.004653,0.150762,0.004653,0.009306,0.004653,0.348986,0.087537,0.004653
...,...,...,...,...,...,...,...,...,...
4150,0.923799,0.016496,0.397563,0.016496,0.016496,0.164964,1.649641,0.016496,0.016496
4151,2.470434,0.083744,1.277089,0.041872,0.041872,0.418718,1.831890,1.046794,0.041872
4152,0.032942,0.000471,0.015953,0.000471,0.001412,0.004706,0.047060,0.000000,0.000471
4153,15.951310,0.306756,9.417408,0.306756,0.306756,3.067560,9.586124,14.417530,0.306756


In [ ]:
def calculate_weighted_mean(df, variable, weights_column='weights'):
    weighted_mean = np.average(df[variable], weights=df[weights_column])
    return weighted_mean

# Control group (Real one)
mean_age_control = df['Age'].mean()
mean_gender_control = df['Gender'].mean()
mean_bmi_control = df['BMI'].mean()
mean_depression_control = df['Depression'].mean()
mean_employment_status_control = df['Employment_status'].mean()
mean_quality_of_life_control = df['Quality_of_life'].mean()
mean_pain_control = df['Pain'].mean()
mean_PA_control = df['Physical_Activity'].mean()

# Control group (with weights)
weighted_mean_age_control = calculate_weighted_mean(df, 'Age')
weighted_mean_gender_control = calculate_weighted_mean(df, 'Gender')
weighted_mean_bmi_control = calculate_weighted_mean(df, 'BMI')
weighted_mean_depression_control = calculate_weighted_mean(df, 'Depression')
weighted_mean_employment_status_control = calculate_weighted_mean(df, 'Employment_status')
weighted_mean_quality_of_life_control = calculate_weighted_mean(df, 'Quality_of_life')
weighted_mean_pain_control = calculate_weighted_mean(df, 'Pain')
weighted_mean_PA_control = calculate_weighted_mean(df, 'Physical_Activity')

# Treatment group (unweighted)
mean_age_treatment = dfg['Age'].mean()
mean_gender_treatment = dfg['Gender'].mean()
mean_bmi_treatment = dfg['BMI'].mean()
mean_Depression_treatment = dfg['Depression'].mean()
mean_employment_status_treatment = dfg['Employment_status'].mean()
mean_QOL_treatment = dfg['Quality_of_life'].mean()
mean_pain_treatment = dfg['Pain'].mean()
mean_PA_treatment = dfg['Physical_Activity'].mean()

# Result
print("Mean in control group:")
print(f"Mean Age (Control Group): {mean_age_control}")
print(f"Mean Gender (Control Group): {mean_gender_treatment}")
print(f"Mean BMI (Control Group): {mean_bmi_control}")
print(f"Mean Depression (Control Group): {mean_depression_control}")
print(f"Mean Employment Status (Control Group): {mean_employment_status_control}")
print(f"Mean Physical Activity (Control Group): {mean_PA_control}")
print(f"Mean QOL (Control Group): {mean_quality_of_life_control}")
print(f"Mean Pain (Control Group): {mean_pain_control}")

print("--------------------------------------------------")
print("Mean in treatnement group:")
print(f"Mean Age (Treatment Group): {mean_age_treatment}")
print(f"Mean Gender (Treatment Group): {mean_gender_control}")
print(f"Mean BMI (Treatment Group): {mean_bmi_treatment}")
print(f"Mean Depression (Treatment Group): {mean_Depression_treatment}")
print(f"Mean Employment Status (Treatment Group): {mean_employment_status_treatment}")
print(f"Mean Physical Activity (Treatment Group): {mean_PA_treatment}")
print(f"Mean QOL (Treatment Group): {mean_QOL_treatment}")
print(f"Mean Pain (Treatment Group): {mean_pain_treatment}")

print("--------------------------------------------------")
print("Weighted mean in control group:")
print(f"Weighted Mean Age (Control Group): {weighted_mean_age_control}")
print(f"Weighted Mean Gender (Control Group): {weighted_mean_gender_control}")
print(f"Weighted Mean BMI (Control Group): {weighted_mean_bmi_control}")
print(f"Weighted Mean Depression (Control Group): {weighted_mean_depression_control}")
print(f"Weighted Mean Employment Status (Control Group): {weighted_mean_employment_status_control}")
print(f"Weighted Mean PA (Control Group): {weighted_mean_PA_control}")
print(f"Weighted Mean QOL (Control Group): {weighted_mean_quality_of_life_control}")
print(f"Weighted Mean Pain (Control Group): {weighted_mean_pain_control}")


Mean in control group:
Mean Age (Control Group): 61.07316486161252
Mean Gender (Control Group): 1.7002499013547283
Mean BMI (Control Group): 28.525318892900124
Mean Depression (Control Group): 1.2659446450060168
Mean Employment Status (Control Group): 1.6953068592057763
Mean Physical Activity (Control Group): 5.671239470517449
Mean QOL (Control Group): 68.01494183714401
Mean Pain (Control Group): 15.12756518251103
--------------------------------------------------
Mean in treatnement group:
Mean Age (Treatment Group): 56.69222675259766
Mean Gender (Treatment Group): 1.5812274368231047
Mean BMI (Treatment Group): 29.592791003551234
Mean Depression (Treatment Group): 1.0695777982375378
Mean Employment Status (Treatment Group): 1.1566486913060634
Mean Physical Activity (Treatment Group): 5.7978429567276075
Mean QOL (Treatment Group): 45.055405760883865
Mean Pain (Treatment Group): 47.21938708404577
--------------------------------------------------
Weighted mean in control group:
Weighted

##### Balanced dataframe:

In [ ]:
# Function to create a new DataFrame with rows replicated according to weights
def create_weighted_df(df, weight_column='weights'):
    # Normalize weights to integer values
    max_weight = df[weight_column].max()
    normalized_weights = (df[weight_column] / max_weight * 100).round().astype(int)
    
    # Create a list to store repeated rows
    weighted_rows = []
    
    # Repeat each row according to its normalized weight
    for i, weight in enumerate(normalized_weights):
        repeated_rows = [df.iloc[i]] * weight
        weighted_rows.extend(repeated_rows)
    
    # Convert the list of rows into a new DataFrame
    weighted_df = pd.DataFrame(weighted_rows).reset_index(drop=True)
    return weighted_df

# Create the new weighted DataFrame
weighted_control_df = create_weighted_df(df)


In [ ]:
weighted_control_df.to_csv('weighted_control.csv', index=False)

In [ ]:
df

,Age,Gender,BMI,Depression,Employment_status,Physical_Activity,Quality_of_life,Pain,weights
0,78.0,1.0,27.6,1.0,3.0,1,50.00,4.1250,0.000398
1,76.0,2.0,27.4,1.0,3.0,1,100.00,0.0000,0.001219
2,69.0,1.0,29.8,1.0,3.0,1,81.25,2.0000,0.000585
3,68.0,2.0,30.1,2.0,3.0,1,75.00,19.8125,0.000610
4,76.0,1.0,32.4,1.0,2.0,1,75.00,18.8125,0.004653
...,...,...,...,...,...,...,...,...,...
4150,56.0,1.0,24.1,1.0,1.0,10,100.00,1.0000,0.016496
4151,59.0,2.0,30.5,1.0,1.0,10,43.75,25.0000,0.041872
4152,70.0,1.0,33.9,1.0,3.0,10,100.00,0.0000,0.000471
4153,52.0,1.0,30.7,1.0,1.0,10,31.25,47.0000,0.306756
